In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./data/cheyenne-Aug-6.csv');

df['datetime']= pd.to_datetime(df.Time, unit='s').values
df = df.sort_values(by='Time')

c = df.Time.rolling(window=2).apply(lambda x: (x[1]-x[0])>3600.0, raw=True ).fillna(0)
df['contig'] = np.cumsum(c)

cn = {v:k for v,k in enumerate(df.columns)}

df[cn[4]]  = df[cn[4]].replace(to_replace=0, value=np.nan)
df[cn[5]]  = df[cn[5]].replace(to_replace=0, value=np.nan)
df[cn[8]] = df[cn[8]].replace(to_replace=0, value=np.nan)
df[cn[9]] = df[cn[9]].replace(to_replace=0, value=np.nan)
df[cn[10]] = df[cn[10]].replace(to_replace=0, value=np.nan)
df[cn[17]] = df[cn[17]].replace(to_replace=0, value=np.nan).interpolate()
df[cn[19]] = df[cn[19]].replace(to_replace=0, value=np.nan).interpolate()

Data consists of two nearly contigious monitoring periords spanning 388 and 4 days respectivly. 

In [3]:
cn

{0: 'Time',
 1: 'Main Engine Fuel Consumption (MT/day)',
 2: 'Main Engine Using HFO (bool)',
 3: 'Main Engine Using MGO (bool)',
 4: 'Draft Forward (meters)',
 5: 'Draft Aft (meters)',
 6: 'Draft Mid-1 (meters)',
 7: 'Draft Mid-2 (meters)',
 8: 'Shaft Speed (RPM)',
 9: 'Shaft Torque (kNm)',
 10: 'Shaft Power (kW)',
 11: 'Speed Over Ground (knots)',
 12: 'Speed Through Water (knots)',
 13: 'Heading (degrees)',
 14: 'Rudder Angle (degrees)',
 15: 'Weather Service Apparent Wind Speed (knots)',
 16: 'Weather Service Apparent Wind Direction (degrees from bow)',
 17: 'Weather Service True Wind Speed (knots)',
 18: 'Weather Service True Wind Direction (degrees from north)',
 19: 'Weather Service Temperature (celsius)',
 20: 'Water Depth (meters)',
 21: 'datetime',
 22: 'contig'}

In [4]:
df['trip'] = (df[cn[1]]==0) 
df['trip'] = np.cumsum(df.trip.rolling(window=2,center=False).apply(lambda x: x[1]!=x[0], raw=True))

h = df.groupby(['trip','contig'])[cn[13]].rolling(window=2,center=False).apply(lambda x: x[1]-x[0], raw=True).reset_index()
h = h.rename({'level_2':'index', cn[13]:'heading_delta'},axis=1).set_index('index').drop(['trip','contig'],axis=1)
df = df.join(h)



In [5]:
def normAngle(angle):
    if angle < -180: angle += 360
    elif angle > 180: angle -= 360
    return angle

def theLen(x): return len(x)
def totalDurr(x): return max(x) - min(x)
def active(x): return np.mean(x)>0

df.heading_delta = df.heading_delta.apply(normAngle).fillna(0)

s = df.groupby('trip').agg({cn[1]: active,
                            cn[21]: [totalDurr , theLen, min, max],
                              
                             }).rename({'min':'startTime', 'max':'endTime'},axis=1)
s.columns = s.columns.droplevel(0)
s = s.reset_index()

s = s[s.active & s.totalDurr.apply(lambda x: x.total_seconds()>3*60**2)].drop(['active','theLen'],axis=1)

s=s.drop('totalDurr',axis=1)

df2 = df.merge(s, on='trip', how='inner')
df2 = df2[(df2.datetime!=df2.startTime) & (df2.datetime!=df2.endTime)]
df2 = df2.drop(['startTime','endTime'],axis=1)

df2[cn[4]]=df2.groupby("trip").transform(lambda x: x.fillna(x.mean()))[cn[4]]
df2[cn[5]]=df2.groupby("trip").transform(lambda x: x.fillna(x.mean()))[cn[5]]



In [6]:
df2['trim'] = df2.apply(lambda x: x[cn[4]]-x[cn[5]],axis=1)
df2['rudder'] = df2[cn[14]].apply(lambda x: np.sin(np.deg2rad(x)) )
df2['depth']  = df2[cn[20]].replace(to_replace=0, value=df2[cn[20]].max())
df2['current'] = (df2[cn[14]].apply(normAngle)/df2.heading_delta.replace(0,1)).values
df2['bowWind_sine'] = np.sin(np.deg2rad((df2[cn[18]]- df2[cn[13]]).apply(normAngle)))
df2['bowWind_cos'] = np.cos(np.deg2rad((df2[cn[18]]- df2[cn[13]]).apply(normAngle)))
df2.heading_delta = np.sin(np.deg2rad(df2.heading_delta))
df2['wind_speed'] = df2[cn[17]]


In [9]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model

lr = sklearn.linear_model.LinearRegression()
n = [cn[v] for v in range(8,11,1)]

D = df2[df2[n].isna().sum(1)<1][n].values
X=D[:,0:2]
y=D[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)
lr.fit(X_train, y_train)
score = lr.score(X_test,y_test)
print(score)

X_unk  = df2[df2[cn[10]].isna()][[cn[8],cn[9]]].values
y_pred = lr.predict(X_unk)


df2.iloc[df2[cn[10]].isna().values, 10] = np.atleast_2d(y_pred).T

df2 = df2[df2.trip!=0]

0.9899431344210847


/Users/frjo6001/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [26]:
df2.to_pickle('process.pkl')